In [1]:
using LinearAlgebra
using StaticArrays


In [2]:
#import Pkg; Pkg.add("StaticArrays")

In [3]:
SVector{4,Float32}

SArray{Tuple{4},Float32,1,4}

In [4]:
d = 3
c = 1
key_type = SVector{d,Float32}
val_type = SVector{c,Float32}
lattice = Dict{key_type,val_type}

Dict{SArray{Tuple{3},Float32,1,3},SArray{Tuple{1},Float32,1,1}}

In [562]:
function canonical_vertices(d)
    canonical_coords = zeros(Int,d+1,d+1)
    for k in 1:d+1
        canonical_coords[k,1:d+2-k].=k-1
        canonical_coords[k,d-k+3:end].=k-d-2
    end
    return canonical_coords' # check transpose vs non transpose is correct
end
function elevate(x::Array{Float32,1})
    d= length(x)
    s = [(d+1)/√(i*(i+1)*2/3) for i in 1:d+1] # Scale factor
    elevated = zeros(Float32,d+1)
    elevated[d+1] = -d*x[d]s[d];
    for i in d:-1:2
        elevated[i] = elevated[i + 1] - (i-1)*x[i-1]s[i-1] + (i + 1)*x[i]s[i]
    end
    elevated[1] = elevated[1] + 2x[1]s[1];
    return elevated
end

# function elevate(x::Array{Float32,2})
#     return [elevate(xi) for xi in x]
# end

function closest_0cell(Ex::Array{Float32,1})
    d = length(Ex)-1
    mygreedy = zeros(Int,d+1)
    for i in 1:d+1
        v = Ex[i]/(d+1)
        up = ceil(v)*(d+1);
        down = floor(v)*(d+1);
        if (up - Ex[i] < Ex[i] - down)
            mygreedy[i] = up
        else
            mygreedy[i] = down
        end
    end
    return mygreedy
end

# function closest_0cell(Ex::Array{Float32,2})
#     return [closest_0cell(xi) for xi in Ex]
# end

function ranks(Ex::Array{Float32,1},closest0::Array{Int,1})
    d = length(Ex)-1
    theranks = zeros(Int,d+1)
    coordsum = Int(sum(closest0)/(d+1))
    for i in 1:d
        for j in i+1:d+1
            if (Ex[i] - closest0[i] < Ex[j] - closest0[j])
                theranks[i]+=1
            else
                theranks[j]+=1
            end
        end
    end
    #println(theranks,coordsum)
    
    if (coordsum > 0)
        #sum too large - the point is off the hyperplane.
        #need to bring down the ones with the smallest differential
        for i in 1:d+1
            if (theranks[i] >= d + 1 - coordsum)
                closest0[i] -= d + 1
                theranks[i] += coordsum - (d + 1)
            else
                theranks[i] += coordsum
            end
        end
    elseif (coordsum < 0)
        #sum too small - the point is off the hyperplane
        #need to bring up the ones with largest differential
        for i in 1:d+1
            if (theranks[i] < -coordsum)
                closest0[i] += d + 1
                theranks[i] += (d + 1) + coordsum
            else
                theranks[i] += coordsum
            end
        end
    end
    #println(theranks)
    return theranks
end

# function ranks(Ex::Array{Float32,2},closest0::Array{Float32,2})
#     return [ranks(xi) for xi in Ex]
# end


function barycentric_weights(Ex::Array{Float32,1},permutation::Array{Int,1},
        closest_vertex::Array{Int,1})
    d = length(Ex)-1
    weights = zeros(Float32,d+2)
    for c in 1:d+1
        #println(d+1-permutation[c]+1,permutation)
        
        weights[d-permutation[c]+1] += (Ex[c]-closest_vertex[c])/(d+1)
        weights[d+1-permutation[c]+1] -= (Ex[c]-closest_vertex[c])/(d+1)
    end
    weights[1] += 1.0 + weights[d+2]
    return weights
end



function splat(vals,X)
    n,d = size(X)
    n,vd = size(vals)
    
    canonical_coords = canonical_vertices(d)
    lattice = Dict{Array{Int,1},Array{Float32,1}}() #TODO replace with static arrays
    #weights = zeros(Float32,n,d+1)
    for i in 1:n
        Ex = elevate(X[i,:])
        closest_vertex = closest_0cell(Ex)
        permutation = ranks(Ex,closest_vertex)
        barycentric = barycentric_weights(Ex,permutation,closest_vertex)
        for rem in 1:d+1
            key = zeros(Int,d+1)
            for c in 1:d+1
                key[c] = closest_vertex[c]+canonical_coords[rem,1+permutation[c]]
            end
            val = get(lattice,key,zeros(Float32,vd))
            val .+= (barycentric[rem].*vals[i])
            lattice[key] = val # necessary?
            #println(key,val)
        end
        #weights[i,:] = barycentric
    end
    return lattice
end

function blur(lattice)
    lattice_new = deepcopy(lattice)
    coeffs = Dict(-1=>.5,0=>1.0,1=>.5)
    d=3
    for c in 1:d+1 # along each of the lattice directions
        for (key,val) in lattice
            vd = length(val)
            lattice_new[key] .*= 0
            for nid in -1:1
                neighbor = key.-nid
                #if (c!=d+1)
                    #println(length(neighbor),c)
                neighbor[c] = key[c]+nid*d
                println(neighbor)
                #end
                nval = get(lattice,neighbor,zeros(Float32,vd))
                out = (nval.*coeffs[nid])
                #println(out)
                lattice_new[key] += out
            end
        end
        lattice,lattice_new = lattice_new,lattice # swap the dictionaries
        println(collect(values(lattice)),collect(values(lattice_new)))
    end
    return lattice
end

function slice(lattice,X_out)
    n,d = size(X_out)
    key = zeros(Int,d+1)
    canonical_coords = canonical_vertices(d)
    vd=1
    filtered_out = zeros(Float32,n,vd)
    for i in 1:n
        Ex = elevate(X_out[i,:])
        closest_vertex = closest_0cell(Ex)
        permutation = ranks(Ex,closest_vertex)
        barycentric = barycentric_weights(Ex,permutation,closest_vertex)
        println(barycentric,sum(barycentric[1:end-1]))
        for rem in 1:d+1
            for c in 1:d+1
                key[c] = closest_vertex[c]+canonical_coords[rem,1+permutation[c]]
            end
            lattice_val = lattice[key]#get(lattice,key,zeros(Float32,vd))
            #println(key,lattice_val)
            filtered_out[i,:] .+= barycentric[rem].*lattice_val./(1.0+2.0^(-d))
        end
        #println(filtered_out[i,:])
    end
    return filtered_out
end

function bilateral_filter(Vin,Xin,Xout)
    lattice = splat(Vin,Xin)
    #println(collect(values(lattice)))
    #print(lattice)
    lattice = blur(lattice)
    #println("__________________")
    Vout = slice(lattice,Xout)
    return Vout
end

function bilateral_filter(Vin,X)
    return bilateral_filter(Vin,X,X)
end


function K(X)
    n,d=size(X)
    k = zeros(Float32,n,n)
    for i in 1:n
        for j in 1:n
            k[i,j] = ℯ.^(-.5norm(X[i,:]-X[j,:])^2)
        end
    end
    return k
end

K (generic function with 1 method)

In [567]:
f = rand(Float32,3,1);
X = rand(Float32,3,3);

In [568]:
X.*5

3×3 Array{Float32,2}:
 2.49868  2.10248  0.804151
 3.40334  2.41138  4.42955
 4.10935  4.65909  2.30867

In [569]:
bilateral_filter(f,X.*5)

[19, 0, -2, -19]
[22, -1, -3, -20]
[25, -2, -4, -21]
[23, 0, -2, -19]
[26, -1, -3, -20]
[29, -2, -4, -21]
[13, -1, -6, -4]
[16, -2, -7, -5]
[19, -3, -8, -6]
[13, -5, -10, -4]
[16, -6, -11, -5]
[19, -7, -12, -6]
[13, -1, -6, 0]
[16, -2, -7, -1]
[19, -3, -8, -2]
[19, 0, -6, -19]
[22, -1, -7, -20]
[25, -2, -8, -21]
[23, 0, -14, -11]
[26, -1, -15, -12]
[29, -2, -16, -13]
[13, -5, -6, -4]
[16, -6, -7, -5]
[19, -7, -8, -6]
[23, 4, -2, -19]
[26, 3, -3, -20]
[29, 2, -4, -21]
[27, 4, -14, -11]
[30, 3, -15, -12]
[33, 2, -16, -13]
[23, 0, -18, -11]
[26, -1, -19, -12]
[29, -2, -20, -13]
[27, 0, -14, -11]
[30, -1, -15, -12]
[33, -2, -16, -13]
Array{Float32,1}[[0.0918303], [0.024858344], [0.007259622], [0.043165423], [0.24218537], [0.05194532], [0.00978169], [0.0030180071], [0.18462881], [0.0088286325], [0.09747044], [0.1312226]]Array{Float32,1}[[0.0918303], [0.024858344], [0.007259622], [0.043165423], [0.24218537], [0.05194532], [0.00978169], [0.0030180071], [0.18462881], [0.0088286325], [0.0974704

3×1 Array{Float32,2}:
 0.18214701
 0.115335874
 0.096663944

In [566]:
Kf = K(X.*.5)*f

3×1 Array{Float32,2}:
 0.78518766
 0.8002254
 0.7610327

In [556]:
f =zeros(Float32,3,1); f[3,:] .= 1;
#f = rand(10,1)
Kf = K(X)*f

3×1 Array{Float32,2}:
 0.9101271
 0.72928065
 1.0

In [557]:
filtered_f = bilateral_filter(f,X)

Array{Float32,1}[[0.099686354], [0.0], [0.72597456], [0.0], [0.0], [0.09395021], [0.0803889], [0.0], [0.0], [0.0], [0.0], [0.0]]
Array{Float32,1}[[0.099686354], [0.0], [0.72597456], [0.0], [0.0], [0.09395021], [0.0803889], [0.0], [0.0], [0.0], [0.0], [0.0]]Array{Float32,1}[[0.099686354], [0.0], [0.72597456], [0.0], [0.0], [0.09395021], [0.0803889], [0.0], [0.0], [0.0], [0.0], [0.0]]
Array{Float32,1}[[0.099686354], [0.0], [0.72597456], [0.0], [0.0], [0.09395021], [0.0803889], [0.0], [0.0], [0.0], [0.0], [0.0]]Array{Float32,1}[[0.099686354], [0.0], [0.72597456], [0.0], [0.0], [0.09395021], [0.0803889], [0.0], [0.0], [0.0], [0.0], [0.0]]
Array{Float32,1}[[0.099686354], [0.0], [0.72597456], [0.0], [0.0], [0.09395021], [0.0803889], [0.0], [0.0], [0.0], [0.0], [0.0]]Array{Float32,1}[[0.099686354], [0.0], [0.72597456], [0.0], [0.0], [0.09395021], [0.0803889], [0.0], [0.0], [0.0], [0.0], [0.0]]
Array{Float32,1}[[0.099686354], [0.0], [0.72597456], [0.0], [0.0], [0.09395021], [0.0803889], [0.0],

3×1 Array{Float32,2}:
 0.0
 0.0
 0.4909026

In [544]:
filtered_f./Kf

3×1 Array{Float32,2}:
 0.0
 0.0
 0.40386727

In [545]:
canonical_coords'

5×5 Adjoint{Float32,Array{Float32,2}}:
 0.0   1.0   2.0   3.0   4.0
 0.0   1.0   2.0   3.0  -1.0
 0.0   1.0   2.0  -2.0  -1.0
 0.0   1.0  -3.0  -2.0  -1.0
 0.0  -4.0  -3.0  -2.0  -1.0

In [546]:
1,...,3

LoadError: syntax: invalid identifier name "..."

In [74]:
sum(1:20)

210

In [119]:
d1 = Dict(1 => 2, 3 => 4);
d2 = Dict(1 => 3, 3 => 4);

In [120]:
d2

Dict{Int64,Int64} with 2 entries:
  3 => 4
  1 => 3

In [121]:
d2,d1=d1,d2

(Dict(3 => 4,1 => 2), Dict(3 => 4,1 => 3))

In [122]:
d2

Dict{Int64,Int64} with 2 entries:
  3 => 4
  1 => 2

In [123]:
d1

Dict{Int64,Int64} with 2 entries:
  3 => 4
  1 => 3